<h1>Customer Lifetime Value Analysis<h1>

## Table of Contents
* [Overview](#1)
* [How to Calculate Customer Lifetime Value](#2)
* [Importing Libraries](#3)
* [Exploratory Data Analysis and Data Preprocessing](#4)
    * [Overview of the Dataset](#4-1)
    * [Missing Values](#4-2)
    * [Outliers Detection and Handling](#4-3)
    * [Feature Engineering](#4-4)
    * [Grouping and Aggregating Data](#4-5)
* [Customer Lifetime Value (CLTV)](#5)
    * [Calculate the Average Purchase Value](#5-1)
    * [Calculate the Average Purchase Frequency](#5-2)
    * [Calculate the Customer Value](#5-3)
    * [Calculate the Average Customer Lifespan](#5-4)
    * [Calculate the CLTV](#5-5)
* [Export to Excel](#6)    

<a id="1"></a>
## Overview
The lifetime value of a customer is intended to assess the financial worth of each customer. It gauges the amount your business can reasonably expect to earn from a customer over the entire lifetime of the relationship. The CLTV metric can be used to measure how long it takes a business to recoup the investment required to acquire a new customer and retain them, such as costs associated with sales and marketing efforts. This is critical to the financial health of a business since you want to ensure profitability and continued revenue growth.





<a id="2"></a>
## How to Calculate Customer Lifetime Value

### Step 1: Average Order Value (AOV)
You begin calculating customer lifetime value by understanding how much revenue you generate from the average customer. This is the average order value (AOV).
$$AOV=\frac{Total\ Revenue}{Total\ Number\ of\ Orders}$$

### Step 2: Average Purchase Frequency (APF)
To calculate APF, determine the total sales in the same time period used for average order value. Divide that by the number of unique customers that made a purchase. Unique customers mean if a customer made multiple purchases in the selected time period, you only count them once.
$$APF=\frac{Total\ Orders}{Total\ Customers}$$

### Step 3: Customer Value (CV)
Next, multiply the AOV and APF numbers calculated previously to arrive at customer value (CV). This tells you how much revenue the average customer brings to your business over the selected time period.
$$CV={Average\ Order\ Value\times Average\ Purchase\ Frequency}$$

### Step 4: Average Customer Lifespan (ACL)
The customer’s lifespan is the amount of time between their first purchase and their last before they no longer patronize your business. You can take an average of this timeframe across all your customers to arrive at the average customer lifespan (ACL).
$$ACL=\frac{Sum\ of\ Customer\ Lifespans}{Number\ of\ Customers}$$

### Step 5: Customer lifetime value (CLTV)
Now that you’ve calculated all the pieces needed, it’s time to put them together to derive the customer lifetime value. The simple approach is to multiply customer value by the average lifespan. The lifetime value calculator, in this case, is as follows:
$$CLTV={Customer\ Value\times Average\ Customer\ Lifespan}$$

<a id="3"></a>
## Importing Libraries 

In [1]:
import pandas as pd
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

<a id="4"></a>
## Exploratory Data Analysis and Data Preprocessing

<a id="4-1"></a>
### Overview of the Dataset
### Variables

**Invoice**: Invoice number. The unique number of each transaction, namely the invoice. Aborted operation if it starts with C.

**StockCode**: Product code. Unique number for each product.

**Description**: Product name

**Quantity**: Number of products. It expresses how many of the products on the invoices have been sold.

**InvoiceDate**: Invoice date and time.

**UnitPrice**: Product price (in GBP)

**CustomerID**: Unique customer number

**Country**: The country where the customer lives.

In [2]:
df = pd.read_excel("online_retail_II.xlsx", sheet_name='Year 2010-2011')

In [3]:
df_copy = df.copy()

In [4]:
def info_data():
    print(df_copy.head())
    print(df_copy.shape)
    print(df.info())

info_data()

  Invoice StockCode                          Description  Quantity  \
0  536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1  536365     71053                  WHITE METAL LANTERN         6   
2  536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3  536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4  536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  Price  Customer ID         Country  
0 2010-12-01 08:26:00   2.55      17850.0  United Kingdom  
1 2010-12-01 08:26:00   3.39      17850.0  United Kingdom  
2 2010-12-01 08:26:00   2.75      17850.0  United Kingdom  
3 2010-12-01 08:26:00   3.39      17850.0  United Kingdom  
4 2010-12-01 08:26:00   3.39      17850.0  United Kingdom  
(541910, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----       

<a id="4-2"></a>
### Missing Values

In [5]:
df_copy.isna().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [6]:
df_copy.dropna(inplace=True)

<a id="4-3"></a>
### Outliers Detection and Handling

In [7]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [8]:
replace_with_thresholds(df_copy, 'Price')
replace_with_thresholds(df_copy, 'Quantity')

<a id="4-4"></a>
### Feature Engineering

In [9]:
country = dict(df_copy['Country'].value_counts())

In [10]:
for i in country:
    if country[i] < 1000:
        df_copy['Country'] = df_copy['Country'].replace({i: 'others'})

In [11]:
df_copy['Country'].value_counts()

Country
United Kingdom    361878
Germany             9495
France              8492
EIRE                7485
others              6805
Spain               2533
Netherlands         2371
Belgium             2069
Switzerland         1877
Portugal            1480
Australia           1259
Norway              1086
Name: count, dtype: int64

In [12]:
df_copy = df_copy[~df_copy['Invoice'].str.contains('C', na=False)]

In [13]:
df_copy = df_copy[df_copy['Price'] > 0]

In [14]:
df_copy['TotalPrice'] = df_copy.Price * df_copy.Quantity

<a id="4-5"></a>
### Grouping and Aggregating Data

In [15]:
df_copy['InvoiceDate'].max()


Timestamp('2011-12-09 12:50:00')

In [16]:
df_copy['InvoiceDate'].min()

Timestamp('2010-12-01 08:26:00')

In [17]:
today_date = dt.datetime(2011, 12, 11)

In [18]:
df_copy = df_copy.groupby(['Customer ID', 'Country']).agg({'TotalPrice':'sum',
                              'InvoiceDate': [lambda date: (date.max()-date.min()).days,
                                              lambda date: (today_date - date.max()).days],
                              'Invoice': lambda Invoice: Invoice.nunique()})

In [19]:
df_copy.columns = ['monetary', 'CustomerLifeSpan','recency', 'frequency']
df_copy = df_copy.reset_index()

In [20]:
df_copy = df_copy[df_copy['frequency'] > 1]
df_copy['recency'] = df_copy['recency'] / 7
df_copy['CustomerLifeSpan'] = df_copy['CustomerLifeSpan'] / 7

<a id="5"></a>
## Customer Lifetime Value (CLTV)

<a id="5-1"></a>
### Calculate the Average Purchase Value:

In [21]:
df_copy['Average Purchase Value'] = df_copy['monetary'] / df_copy['frequency']

<a id="5-2"></a>
### Calculate the Average Purchase Frequency:

In [22]:
df_copy['Average purchase frequency'] = df_copy['frequency'].sum() / df_copy['Customer ID'].nunique()

<a id="5-3"></a>
### Calculate the Customer Value:

In [23]:
df_copy['Customer Value'] = df_copy['Average Purchase Value'] * df_copy['Average purchase frequency']

<a id="5-4"></a>
### Calculate the Average Customer Lifespan:

In [24]:
df_copy['Average customer lifespan'] = df_copy['CustomerLifeSpan'].sum() / df_copy['Customer ID'].nunique()

<a id="5-5"></a>
### Calculate the CLTV:

In [25]:
df_copy['CLTV'] = df_copy['Customer Value'] * df_copy['Average customer lifespan']

In [26]:
df_copy

,Customer ID,Country,monetary,CustomerLifeSpan,recency,frequency,Average Purchase Value,Average purchase frequency,Customer Value,Average customer lifespan,CLTV
1,12347.0,others,4310.000,52.142857,0.428571,7,615.714286,5.989803,3688.007334,28.423649,104826.625168
2,12348.0,others,1771.905,40.285714,10.857143,4,442.976250,5.989803,2653.340513,28.423649,75417.618845
5,12352.0,Norway,1757.590,37.142857,5.285714,8,219.698750,5.989803,1315.952253,28.423649,37404.164644
9,12356.0,Portugal,2811.430,43.142857,3.285714,3,937.143333,5.989803,5613.304038,28.423649,159550.582489
11,12358.0,others,1151.170,21.285714,0.285714,2,575.585000,5.989803,3447.640814,28.423649,97994.531632
...,...,...,...,...,...,...,...,...,...,...,...
4333,18272.0,United Kingdom,3078.580,34.857143,0.428571,6,513.096667,5.989803,3073.348002,28.423649,87355.764191
4334,18273.0,United Kingdom,204.000,36.428571,0.428571,3,68.000000,5.989803,407.306610,28.423649,11577.140042
4341,18282.0,United Kingdom,178.050,16.857143,1.142857,2,89.025000,5.989803,533.242220,28.423649,15156.689591
4342,18283.0,United Kingdom,2094.880,47.571429,0.571429,16,130.930000,5.989803,784.244919,28.423649,22291.102142


<a id="6"></a>
## Export to Excel

In [27]:
df_copy.to_excel("CLTV-analysis.xlsx", index=False)